# Feature Engineering for Water Quality Clustering

## Executive Summary

This notebook details the feature engineering pipeline designed to prepare the cleaned water quality dataset for unsupervised clustering. The primary objective is to transform the raw variables into a production-ready, numerical feature set optimized for a K-Means model.

The strategy is guided by the findings from the `EDA.ipynb` notebook, which highlighted several key data characteristics:
1.  **High Cardinality:** Categorical location features like `MUNICIPIO` and `ACUIFERO` are too complex for direct use.
2.  **Redundancy:** The dataset contains redundant information, such as ID columns, constant columns, and pairs of numerical measurements with their derived quality labels.
3.  **Distribution Skew:** Many numerical measurements exhibit a strong right-skew, which can negatively impact distance-based algorithms like K-Means.
4.  **Multicollinearity:** High correlation was noted between variables related to salinity and dissolved solids.

The following steps will systematically address these issues to produce a final, scaled, and robust feature set for modeling.

### Step 1: Initial Feature Selection (Exclusion of Uninformative Columns)

**Justification:** The first step is to remove columns that provide no value for a clustering model or introduce noise. This includes identifiers, constants, high-cardinality categoricals, and redundant derived labels.

1.  **Identifier Columns (`CLAVE`, `SITIO`):** These are unique IDs for each monitoring site and have no predictive power. Their high cardinality (1068 and 1066 unique values, respectively) would add noise.
2.  **Constant Column (`PERIODO`):** This column has only one value (2020) and thus offers no variance or information to the model.
3.  **High-Cardinality Categorical Columns (`ORGANISMO_DE_CUENCA`, `ESTADO`, `MUNICIPIO`, `ACUIFERO`, `SUBTIPO`):** As identified in the EDA, `MUNICIPIO` (452 unique values) and `ACUIFERO` (273 unique values) are too complex to be one-hot encoded. The geographic information is better and more efficiently represented by the `LONGITUD` and `LATITUD` columns.
4.  **Redundant Categorical/Compliance Columns (`CALIDAD_*`, `CUMPLE_CON_*`, `SEMAFORO`):** These columns are derived directly from the numerical measurements (e.g., `CALIDAD_AS` is derived from `AS_TOT_mg/L`). Including both the raw measurement and its categorical label is redundant. We will retain the numerical columns as they contain more granular information, which is ideal for clustering.
5.  **Multi-Label Text Column (`CONTAMINANTES`):** This column summarizes which numerical features exceed a certain threshold. It is redundant information already captured by the numerical columns themselves and will be excluded.

In [ ]:
import pandas as pd

# This cell assumes 'df_cleaned' is the pre-processed DataFrame from the EDA notebook
# In a real pipeline, you would load it: df_cleaned = pd.read_csv('path/to/cleaned_data.csv')

def initial_feature_selection(df):
    """Removes uninformative, redundant, and high-cardinality columns.""""
    # Identify all 'CALIDAD_*' and 'CUMPLE_CON_*' columns dynamically
    quality_cols = [col for col in df.columns if col.startswith('CALIDAD_')]
    compliance_cols = [col for col in df.columns if col.startswith('CUMPLE_CON_')]

    cols_to_drop = [
        # 1. Identifiers
        'CLAVE', 'SITIO',
        # 2. Constants
        'PERIODO',
        # 3. High-Cardinality Categoricals
        'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO', 'ACUIFERO', 'SUBTIPO',
        # 4. Redundant Categorical/Compliance Columns
        'SEMAFORO', 
        *quality_cols,
        *compliance_cols,
        # 5. Redundant Multi-Label Text
        'CONTAMINANTES'
    ]
    
    # Ensure columns exist before dropping to prevent errors
    cols_to_drop_existing = [col for col in cols_to_drop if col in df.columns]
    
    df_selected = df.drop(columns=cols_to_drop_existing)
    
    print("Columns dropped:", len(cols_to_drop_existing))
    print("Remaining columns:", df_selected.shape[1])
    print("--- Final Selected Features ---")
    print(df_selected.columns.tolist())
    
    return df_selected

# Example usage (assuming df_cleaned is available)
# features_selected = initial_feature_selection(df_cleaned)

### Step 2: Logarithmic Transformation of Skewed Features

**Justification:** The EDA revealed that most numerical water quality measurements are heavily right-skewed. Distance-based algorithms like K-Means are sensitive to skewed distributions and outliers, as they can dominate the distance calculations. Applying a logarithmic transformation (`log1p`, which handles zero values) compresses the range of these variables, reduces the impact of extreme outliers, and makes the distributions more symmetric, leading to more stable and meaningful clusters.

In [ ]:
import numpy as np

def apply_log_transformation(df):
    """Applies a log1p transformation to all numerical columns except lat/long."""
    df_transformed = df.copy()
    
    # Exclude geographic coordinates from the transformation
    cols_to_transform = [col for col in df_transformed.columns if col not in ['LONGITUD', 'LATITUD']]
    
    for col in cols_to_transform:
        df_transformed[col] = np.log1p(df_transformed[col])
        
    print(f"Log1p transformation applied to {len(cols_to_transform)} columns.")
    
    return df_transformed

# Example usage:
# features_log_transformed = apply_log_transformation(features_selected)

### Step 3: Scaling Features

**Justification:** The final step before modeling is to scale the features. K-Means calculates distances between data points, so variables with larger scales and variances (e.g., `CONDUCT_mS/cm`) would have a disproportionately large influence on the cluster assignments compared to variables with smaller scales (e.g., `AS_TOT_mg/L`). Using `StandardScaler` standardizes each feature by subtracting the mean and dividing by the standard deviation. This ensures that all features have a mean of 0 and a standard deviation of 1, giving them equal weight in the clustering process.

In [ ]:
from sklearn.preprocessing import StandardScaler

def scale_features(df):
    """Scales all features using StandardScaler."""
    scaler = StandardScaler()
    
    # The scaler returns a numpy array, so we convert it back to a DataFrame
    scaled_features_array = scaler.fit_transform(df)
    df_scaled = pd.DataFrame(scaled_features_array, columns=df.columns, index=df.index)
    
    print("All features have been scaled using StandardScaler.")
    print("--- Final Production-Ready Features ---")
    display(df_scaled.head())
    
    return df_scaled, scaler

# Example usage:
# final_features, scaler_object = scale_features(features_log_transformed)

### Step 4: Discretization (Binning) of Numerical Features

**Justification:** As an alternative to direct scaling and transformation, we can bin numerical features. This approach converts continuous variables into categorical ones, which can be beneficial for several reasons:
1.  **Handles Outliers:** Extreme values are grouped into the highest or lowest bin, reducing their skewing effect.
2.  **Captures Non-Linearity:** It allows models (especially linear ones) to learn non-linear patterns in the data.
3.  **Improves Robustness:** The model becomes less sensitive to small fluctuations in the input data.

We will use quantile-based binning (`pd.qcut`), which divides the data into bins with an equal number of observations, making it robust to skewed distributions.

In [ ]:
def apply_binning(df, n_bins=5):
    """Applies quantile-based binning to numerical columns and returns a one-hot encoded DataFrame."""
    df_binned = df.copy()
    
    # Identify numerical columns to bin, excluding geographic coordinates
    cols_to_bin = [col for col in df_binned.columns if col not in ['LONGITUD', 'LATITUD']]
    
    binned_dfs = []
    
    for col in cols_to_bin:
        # Create bins and one-hot encode them
        binned_col = pd.qcut(df_binned[col], q=n_bins, labels=False, duplicates='drop')
        one_hot_binned = pd.get_dummies(binned_col, prefix=f"{col}_bin")
        binned_dfs.append(one_hot_binned)
    
    # Combine the one-hot encoded bins with the original lat/long
    df_final_binned = pd.concat([df_binned[['LONGITUD', 'LATITUD']]] + binned_dfs, axis=1)
    
    print(f"Applied {n_bins}-quantile binning to {len(cols_to_bin)} columns.")
    print(f"Resulting feature space has {df_final_binned.shape[1]} columns after one-hot encoding.")
    
    return df_final_binned

# Example Usage:
# features_binned = apply_binning(features_selected, n_bins=5)

### Full Pipeline Execution

This cell combines all the above steps into a single, executable pipeline to transform the cleaned data into the final, model-ready feature set.

In [ ]:
def create_feature_pipeline(df_cleaned, use_binning=False):
    """Executes the full feature engineering pipeline."""
    print("--- Starting Feature Engineering Pipeline ---\n")
    
    # Step 1: Initial Feature Selection (common to both paths)
    print("Step 1: Performing initial feature selection...")
    features_selected = initial_feature_selection(df_cleaned)
    print("\n--------------------------------------------\n")
    
    if use_binning:
        # Path 2: Binning and One-Hot Encoding
        print("Step 2 (Alternative): Applying quantile-based binning...")
        final_features = apply_binning(features_selected, n_bins=5)
        scaler = None # No scaler needed for this path
    else:
        # Path 1: Log Transformation and Scaling (for distance-based models)
        print("Step 2: Applying logarithmic transformation...")
        features_log_transformed = apply_log_transformation(features_selected)
        print("\n--------------------------------------------\n")
        
        print("Step 3: Scaling features...")
        final_features, scaler = scale_features(features_log_transformed)

    print("\n--- Feature Engineering Pipeline Complete ---")
    
    return final_features, scaler

# To run the pipeline, you would need the 'df_cleaned' DataFrame from the EDA.
# For example:
# df_cleaned = pd.read_csv('data/cleaned_water_quality_2020.csv')

# To get log-transformed and scaled features (for K-Means):
# final_kmeans_features, production_scaler = create_feature_pipeline(df_cleaned, use_binning=False)

# To get binned and one-hot encoded features (for other models):
# final_binned_features, _ = create_feature_pipeline(df_cleaned, use_binning=True)